<a href="https://colab.research.google.com/github/quentin-arzalier/tp-information-retrieval-with-llm-student-version/blob/main/1-Recherche%20d'information%20classique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1. - Recherche d'Information classique

Dans cette partie, nous allons mettre en oeuvre des principes et modèles classiques de Recherche d'Information. Le jeu de donnée est une collection de livres au format texte (.txt) de Henry Rider Haggard. Jetez un oeil à ces documents dans le dossier _data_.

En sortie de ce module, vous serez capable de :

- Construire un index inversé ;
- Effectuer des requêtes simples selon le modèle booléen :
- Calculer la pondération des termes selon la méthode TF-IDF ;
- Mettre en oeuvre le modèle vectoriel de recherche d'information et y appliquer des requêtes.

### Import des bibliothèques logicielles et configuration

Les lignes suivantes permettent d'instancier un objet la classe `IRSystem` représentant notre moteur de recherche et de charger les données en RAM.

In [4]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/quentin-arzalier/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

Cloning into 'tp-information-retrieval-with-llm-student-version'...
remote: Enumerating objects: 2253, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 2253 (delta 24), reused 29 (delta 15), pack-reused 2212
Receiving objects: 100% (2253/2253), 13.96 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/tp-information-retrieval-with-llm-student-version/tp-information-retrieval-with-llm-student-version


#### Chargement des données

Les lignes ci-dessous permettent de charger les données qui sont un ensemble de 60 livres au format texte (.txt) d'[Henry Rider Haggard ](https://fr.wikipedia.org/wiki/Henry_Rider_Haggard).


In [5]:
from classic_ir.IRSystem import *

# !rm -rf ./data/RiderHaggard/stemmed
ir_system = IRSystem()
ir_system.read_data('./data/RiderHaggard') # chargement des données et prétraitement des documents (stemming).

Reading in documents...
Stemming Documents...
The Ivory Child 2841.txt
    Doc 1 of 60: The Ivory Child
Eric Brighteyes 2721.txt
    Doc 2 of 60: Eric Brighteyes
Red Eve 3094.txt
    Doc 3 of 60: Red Eve
Jess 5898.txt
    Doc 4 of 60: Jess
Stories by English Authors Africa (Selected by Scribners) 1980.txt
    Doc 5 of 60: Stories by English Authors Africa (Selected by Scribners)
Love Eternal 3709.txt
    Doc 6 of 60: Love Eternal
Child of Storm 1711.txt
    Doc 7 of 60: Child of Storm
Marie An Episode in The Life of the late Allan Quatermain 1690.txt
    Doc 8 of 60: Marie An Episode in The Life of the late Allan Quatermain
The Wizard 2893.txt
    Doc 9 of 60: The Wizard
Ayesha, the Return of She 5228.txt
    Doc 10 of 60: Ayesha, the Return of She
Lysbeth, a Tale of the Dutch 5754.txt
    Doc 11 of 60: Lysbeth, a Tale of the Dutch
The Wanderer's Necklace 3097.txt
    Doc 12 of 60: The Wanderer's Necklace
The Lady of Blossholme 3813.txt
    Doc 13 of 60: The Lady of Blossholme
Black He

### Exercice 1. - Construction de l'index inversé

Ce premier exercice a pour objectif de construire l'index inversé non positionnel. L'attribut `self.inverted_index` est un tableau associatif qui associe une liste d'entiers (docIDs) à un mot (word).

Documentation ici https://docs.python.org/3/library/collections.html#collections.defaultdict.

Exercice : modifier la fonction `index` pour calculer l'index inversé.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [6]:
# Exercice 1. Indexation

def index(self):
    """
    Construit l'index inversé et le stocke dans self.inverted_index.

    Dans le code ci-dessous, seul le dictionnaire des tokens est construit. Les postings lists sont vides.
    """
    print("Indexing...")
    self.tf = defaultdict(Counter) # tf est un dictionnaire qui à un document associe un Counter de mots.
    inverted_index = defaultdict(list) # inverted_index est un dictionnaire qui à un mot associe une liste de documents.
    for i in range (len(self.docs)):
        for word in self.docs[i]:
            if (not i in inverted_index[word]):
              inverted_index[word].append(i)

    self.inverted_index = inverted_index

# Ne pas modifier les lignes ci-dessous
IRSystem.index = index
ir_system.index()
run_tests(ir_system, part=0)

Indexing...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 2. - Recherche booléenne

Ce deuxième exercice a pour objectif d'implémenter la recherche booléenne. La requête `query` est une liste de mots _lemmatisés_ et l'algorithme doit rechercher les documents qui contiennent TOUS ces mots.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.


Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [7]:
# Exercice 2. Recherche booléenne
def boolean_retrieve(self, query):
    print(len(self.vocab))
    """
    A partir d'une requête sous la forme d'une liste de mots *lemmatisés*,
    retourne la liste des documents dans lesquels *tous* ces mots apparaissent (ie une requête AND).
    Retourne une liste vide si la requête ne retourne aucun document.

    Dans le code ci-dessous, tous les documents répondent.
    """
    docs = list()
    for idx, doc in enumerate(self.docs):
        test=True
        i=0
        while(test and i < len(query)):
          test=(query[i] in doc)
          i+=1
        if (test):
          docs.append(idx)
    return docs

# Ne pas modifier les lignes ci-dessous
IRSystem.boolean_retrieve = boolean_retrieve
run_tests(ir_system, part=1)

===== Running tests =====
Boolean Retrieval Test
46735
46735
46735
46735
46735
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 3. - Calcul des poids TF-IDF des termes dans les documents

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours. Utiliser le logarithme en base 10.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [8]:
from math import log10

# Exercice 3. calcul des scores tf-idf
def compute_tfidf(self):
    """
    Calcule les scores tf-idf pour tous les mots de tous les documents et les stocke dans self.tfidf.

    Dans le code ci-dessous, les scores tf-idf sont tous nuls.
    """
    print("Calculating tf-idf...")

    N = len(self.docs)  # N = nombre de documents

    occurences = defaultdict(Counter)
    for i in range(N):
      for word in self.docs[i]:
        occurences[i][word] += 1


    self.tfidf = defaultdict(Counter)
    for i in range(N):
      for idx, word in enumerate(self.docs[i]):
        df = len(self.inverted_index[word])
        idf = log10(N/df)
        try:
            self.tfidf[i][word] = (1+log10(occurences[i][word]))*idf
        except ValueError:
            self.tfidf[i][word] = 0.

    print(self.tfidf[0][self.p.stem("separation")])

# Ne pas modifier les lignes ci-dessous
IRSystem.compute_tfidf = compute_tfidf
ir_system.compute_tfidf()
run_tests(ir_system, part=2)

Calculating tf-idf...
0.13337736503241182
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 4. - Calcul de la similarité cosinus.

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours en considérant les éléments suivants :
- Ne considérer que la mesure TF pour calculer le poids des termes de la requête (on utilise pas IDF).
- utiliser le logarithme en base 10.

Exercice : modifier la fonction `rank_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [10]:
from math import sqrt

# Exercice 4. Similarité cosinus
def rank_retrieve(self, query):
    """
    A partir d'une requête (une liste de mots), retourne une liste de documents (classés par docID) et de scores pour la requête en appliquant la simalirité cosinus.

    Dans l'exemple ci-dessous. C'est la mesure de Jaccard qui est utilisée.
    """

    query_set = set(query)

    relevantDocs = set()

    #D'après l'algorithme décrit en slide 58 du cours, ceci devrait donner la bonne réponse :

    # 1 float Scores[N] = 0
    scores = [0.0 for _ in range(len(self.docs))]
    # 2 float Length[N]
    length = [0.0 for _ in range(len(self.docs))]

    # 4 do calculate wt,q and fetch postings list for t
    # 6 for each pair (d, tft,d) in postings list
    for d in range(len(self.docs)):
        sumTfSquared = 0
        # 3 for each query term t
        for q in query:
          occ = self.docs[d].count(q)
          # calcul de wt,d, wt,q toujours 1 car une seule occurence dans query
          tf = 1+log10(occ) if occ > 0 else 0
          # 6 do Scores[d] += wtd x wtq (encore une fois, wtq = 1)
          scores[d] += tf
          sumTfSquared += tf**2
        # 7 Read the array Length (Racine des tf de la query pour le doc au ²)
        length[d] = sqrt(sumTfSquared)

    # 8 for each d
    for d in range(len(self.docs)):
      # 9 do Scores[d] = Scores[d]/Length[d]
      scores[d] = scores[d]/length[d] if length[d] > 0 else 0

    # Je ne comprends pas pourquoi l'algo tape à côté de la bonne réponse.


    # Tri des scores
    ranking = [idx for idx, sim in sorted(enumerate(scores),
                                        key=lambda pair: pair[1], reverse=True)]
    results = []
    for i in range(10):
        results.append((ranking[i], scores[ranking[i]]))

    print(query, results)
    return results

# Ne pas modifier les lignes ci-dessous
IRSystem.rank_retrieve = rank_retrieve
run_tests(ir_system, part=3)

===== Running tests =====
Cosine Similarity Test
['separ', 'of', 'church', 'and', 'state'] [(38, 2.1419726827862604), (0, 2.136789225653016), (25, 2.1349605665697906), (18, 2.1304899715038657), (14, 2.1225348117779252), (55, 2.1216295185335374), (42, 2.1193139986367604), (13, 2.1109427924451), (46, 2.1085493754179545), (54, 2.1068643980705444)]
['priestess', 'ritual', 'sacrific'] [(3, 1.697899637716011), (59, 1.683763952478558), (36, 1.6739563510462148), (40, 1.6650534297328747), (58, 1.558137057486278), (1, 1.4135443130966079), (16, 1.41267430767582), (47, 1.4113812403907187), (48, 1.410300030712635), (42, 1.4022646449955054)]
['demon', 'versu', 'man'] [(39, 1.5825325678221915), (15, 1.5394528176145217), (41, 1.4864590856994584), (29, 1.3744513994264207), (16, 1.3609178024320054), (6, 1.3588434813391523), (40, 1.3393022834787274), (20, 1.3313089586145068), (43, 1.3277687910252949), (14, 1.3249841719678033)]
['african', 'queen', 'marriag'] [(22, 1.72690201800696), (35, 1.72523392557899